In [1]:
from efficientnet_pytorch import EfficientNet


# In[2]:


import torch
import torch.nn as nn
import torch.nn.functional as F


# In[3]:


from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from torch.optim import Optimizer


In [2]:
 model_name = 'efficientnet-b5'


In [3]:
image_size = EfficientNet.get_image_size(model_name)


model = EfficientNet.from_pretrained(model_name)

np.random.seed(13)

src = (ImageList.from_folder(path='fastai-data').split_by_rand_pct(0.2).label_from_folder())

object_detection_results_path = os.getcwd()+"\\result-of-snakes-object-detection" #this folder has the results of preprocessing, predicts where the snakes are and crops them. Look into porting this to evaluator as well

src.add_test_folder(object_detection_results_path)

Loaded pretrained weights for efficientnet-b5


LabelLists;

Train: LabelList (1152 items)
x: ImageList
Image (3, 500, 375),Image (3, 500, 375),Image (3, 334, 500),Image (3, 500, 375),Image (3, 375, 500)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: fastai-data;

Valid: LabelList (288 items)
x: ImageList
Image (3, 375, 500),Image (3, 500, 375),Image (3, 421, 500),Image (3, 500, 375),Image (3, 500, 375)
y: CategoryList
class-872,class-639,class-508,class-653,class-629
Path: fastai-data;

Test: LabelList (17686 items)
x: ImageList
Image (3, 1730, 777),Image (3, 324, 329),Image (3, 178, 264),Image (3, 1267, 1343),Image (3, 178, 362)
y: EmptyLabelList
,,,,
Path: fastai-data

In [5]:
tfms=([rotate(degrees=(-90,90), p=0.8)],[])

bs=8
data = (src.transform(tfms, size=image_size, resize_method=ResizeMethod.SQUISH)
    .databunch(bs=bs).normalize(imagenet_stats))
model.add_module('_fc',nn.Linear(2048, data.c)) #Replace the final layer of b5 model with number of classes
loss_func =LabelSmoothingCrossEntropy() #following EfficientNet paper
RMSprop = partial(torch.optim.RMSprop) #Following EfficientNet paper

learn = Learner(data, model, loss_func=loss_func, opt_func=RMSprop, metrics=[accuracy,FBeta(beta=1,average='macro')])

learn.split([[learn.model._conv_stem, learn.model._bn0, learn.model._blocks[:19]],
         [learn.model._blocks[19:],learn.model._conv_head], 
         [learn.model._bn1,learn.model._fc]]) #for differential learning

learn.load('b5-seed-13-round-3') #best single model - 85.2 local cv. Try ensemble later. with 83.1 and 85.2 models

Learner(data=ImageDataBunch;

Train: LabelList (1152 items)
x: ImageList
Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456)
y: CategoryList
class-1059,class-1059,class-1059,class-1059,class-1059
Path: fastai-data;

Valid: LabelList (288 items)
x: ImageList
Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456)
y: CategoryList
class-872,class-639,class-508,class-653,class-629
Path: fastai-data;

Test: LabelList (17686 items)
x: ImageList
Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456)
y: EmptyLabelList
,,,,
Path: fastai-data, model=EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    

In [12]:
df_test = pd.read_csv('given_submission_sample_file.csv',low_memory=False) #use the given sample file to replace probabilites with our model predictions, This way, no need to worry about corrupted images


df_classes = pd.read_csv('class.csv',low_memory=False) #class mapping

data_dict = df_classes.set_index('class_idx')['original_class'].to_dict()

In [13]:
data_dict

{4: 'thamnophis_proximus',
 67: 'nerodia_sipedon',
 72: 'opheodrys_vernalis',
 78: 'crotalus_horridus',
 128: 'crotalus_pyrrhus',
 140: 'nerodia_rhombifer',
 204: 'thamnophis_sirtalis',
 239: 'natrix_natrix',
 273: 'crotalus_adamanteus',
 326: 'charina_bottae',
 337: 'pituophis_catenifer',
 362: 'lampropeltis_triangulum',
 390: 'nerodia_erythrogaster',
 394: 'thamnophis_marcianus',
 448: 'lampropeltis_californiae',
 450: 'crotalus_ruber',
 460: 'rhinocheilus_lecontei',
 508: 'opheodrys_aestivus',
 526: 'thamnophis_ordinoides',
 536: 'thamnophis_radix',
 540: 'masticophis_flagellum',
 543: 'pantherophis_vulpinus',
 561: 'hierophis_viridiflavus',
 581: 'heterodon_platirhinos',
 597: 'pantherophis_emoryi',
 629: 'regina_septemvittata',
 635: 'haldea_striatula',
 639: 'diadophis_punctatus',
 653: 'nerodia_fasciata',
 654: 'storeria_occipitomaculata',
 663: 'crotalus_scutulatus',
 697: 'storeria_dekayi',
 707: 'crotalus_viridis',
 734: 'boa_imperator',
 771: 'pantherophis_obsoletus',
 784: 